In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp -r '/content/drive/My Drive/Colab Notebooks/[Kaggle] Understanding Clouds from Satellite Images/Scripts/.' .

In [0]:
!unzip -q '/content/drive/My Drive/Colab Notebooks/[Kaggle] Understanding Clouds from Satellite Images/Data/train_images320x480.zip'

### Dependencies

In [4]:
from utillity_script_cloud_segmentation import *

Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [0]:
seed = 0
seed_everything(seed)
warnings.filterwarnings("ignore")

In [0]:
base_path = '/content/drive/My Drive/Colab Notebooks/[Kaggle] Understanding Clouds from Satellite Images/'
data_path = base_path + 'Data/'
model_base_path = base_path + 'Models/files/classification/'
model_path = model_base_path + '18-EfficientNetB0_224x224_OneCycle.h5'
hold_out_set_path = data_path + 'hold-out.csv'
train_images_dest_path = 'train_images/'

### Load data

In [7]:
hold_out_set = pd.read_csv(hold_out_set_path)
X_val = hold_out_set[hold_out_set['set'] == 'validation']
print('Validation samples: ', len(X_val)) 

# Preprocecss data
label_columns=['Fish', 'Flower', 'Gravel', 'Sugar']
for label in label_columns:
  X_val[label].replace({0: 1, 1: 0}, inplace=True)

display(X_val.head())

Validation samples:  1105


,image,Fish_mask,Flower_mask,Gravel_mask,Sugar_mask,Fish,Flower,Gravel,Sugar,set
4420,43dda13.jpg,NaN,2163025 1187 2164425 1187 2165825 1187 2167225...,63503 618 64903 618 66303 618 67703 618 69103 ...,36613 1185 38013 1185 39413 1185 40813 1185 42...,1,0,0,0,validation
4421,3624c8b.jpg,NaN,1209169 427 1210569 427 1211969 427 1213369 42...,11404 414 12804 414 14204 414 15604 414 17004 ...,24824 376 26224 376 27624 376 29024 376 30424 ...,1,0,0,0,validation
4422,519820d.jpg,293243 664 294643 664 296043 664 297443 664 29...,NaN,NaN,595022 308 595331 2 595336 2 595340 2 595344 3...,0,1,1,0,validation
4423,9ede0ba.jpg,860577 8 861977 8 863378 3 863382 6 864777 1 8...,1366803 985 1368203 985 1369603 985 1371003 98...,753290 602 753893 1 753895 4 753901 4 753911 5...,1 1208 1401 1208 2801 1208 4201 1208 5601 1208...,0,0,0,0,validation
4424,99b9bd5.jpg,1362214 400 1363614 400 1365014 400 1366414 40...,NaN,16067 541 17467 541 18867 541 20267 541 21667 ...,428 542 1828 542 3228 542 4628 542 5260 321 60...,0,1,0,0,validation


# Model parameters

In [0]:
HEIGHT = 224
WIDTH = 224
BETA = 0.1

# Model

In [0]:
model = load_model(model_path)

### Classification data generator

In [10]:
test_datagen=ImageDataGenerator(rescale=1./255.)

valid_generator=test_datagen.flow_from_dataframe(
                              dataframe=X_val,
                              directory=train_images_dest_path,
                              x_col="image",
                              y_col=label_columns,
                              target_size=(HEIGHT, WIDTH),
                              batch_size=1,
                              class_mode="other",
                              shuffle=False,
                              seed=seed)

Found 1105 validated image filenames.


# Classification threshold and mask size tunning

In [11]:
valid_preds = model.predict_generator(valid_generator)
best_tresholds = classification_tunning(valid_generator.labels, valid_preds, label_columns, beta=BETA)

Fish treshold=0.87 Score=0.833
Flower treshold=0.98 Score=0.936
Gravel treshold=0.90 Score=0.749
Sugar treshold=0.92 Score=0.752


# Metrics

In [12]:
get_metrics_classification(X_val, valid_preds, label_columns, best_tresholds)

Metrics for: Fish
              precision    recall  f1-score   support

           0       0.52      0.99      0.68       555
           1       0.93      0.07      0.13       550

    accuracy                           0.54      1105
   macro avg       0.73      0.53      0.41      1105
weighted avg       0.72      0.54      0.41      1105

Metrics for: Flower
              precision    recall  f1-score   support

           0       0.54      0.97      0.69       473
           1       0.95      0.37      0.53       632

    accuracy                           0.63      1105
   macro avg       0.74      0.67      0.61      1105
weighted avg       0.77      0.63      0.60      1105

Metrics for: Gravel
              precision    recall  f1-score   support

           0       0.56      0.96      0.71       587
           1       0.78      0.16      0.26       518

    accuracy                           0.58      1105
   macro avg       0.67      0.56      0.49      1105
weighted avg    